In [70]:
import ipywidgets as widgets
import sys
from pathlib import Path
import os
import importlib


module_path='preprocessing/day_intervals_preproc'
if module_path not in sys.path:
    sys.path.append(module_path)

module_path='utils'
if module_path not in sys.path:
    sys.path.append(module_path)
    
module_path='preprocessing/hosp_module_preproc'
if module_path not in sys.path:
    sys.path.append(module_path)
    
module_path='model'
if module_path not in sys.path:
    sys.path.append(module_path)

root_dir = "/data/corpora_alpha/MIMIC/MIMIC_IV_2.2/files"

import day_intervals_cohort
from day_intervals_cohort import *

import day_intervals_cohort_v2
from day_intervals_cohort_v2 import *

import data_generation_icu

import data_generation
import evaluation

import feature_selection_hosp
from feature_selection_hosp import *

# import train
# from train import *


import ml_models
from ml_models import *

import dl_train
from dl_train import *

import tokenization
from tokenization import *


import behrt_train
from behrt_train import *

import feature_selection_icu
from feature_selection_icu import *
import fairness
import callibrate_output

In [71]:
importlib.reload(day_intervals_cohort)
import day_intervals_cohort
from day_intervals_cohort import *

importlib.reload(day_intervals_cohort_v2)
import day_intervals_cohort_v2
from day_intervals_cohort_v2 import *

importlib.reload(data_generation_icu)
import data_generation_icu
importlib.reload(data_generation)
import data_generation

importlib.reload(feature_selection_hosp)
import feature_selection_hosp
from feature_selection_hosp import *

importlib.reload(feature_selection_icu)
import feature_selection_icu
from feature_selection_icu import *

importlib.reload(tokenization)
import tokenization
from tokenization import *

importlib.reload(ml_models)
import ml_models
from ml_models import *

importlib.reload(dl_train)
import dl_train
from dl_train import *

importlib.reload(behrt_train)
import behrt_train
from behrt_train import *

importlib.reload(fairness)
import fairness

importlib.reload(callibrate_output)
import callibrate_output

importlib.reload(evaluation)
import evaluation

In [72]:
def eda_basic(dataframes, dataframes_names = None, log_path=None):
    '''
    Perform basic exploratory data analysis on a list of pandas DataFrames.
    Args:
    dataframes (list): A list of pandas DataFrames.
    dataframes_names (list, optional): A list of names for the DataFrames. If provided, the names will be used to distinguish between the DataFrames in the results.
    log_path (str, optional): The path to save the log file. If provided, the results will be logged to the file.
    '''
    if(len(dataframes) == 0):
        print("No dataframes provided.")
        return
    
    if dataframes_names:
        if len(dataframes) != len(dataframes_names):
            print("The number of dataframes and the number of names provided do not match.")
            return

    if log_path:
        log_file_path = os.path.join(log_path, 'eda_basic_log.txt')
        os.makedirs(os.path.dirname(log_file_path), exist_ok=True)
        log_file = open(log_file_path, 'a')
        log_file.write(f"{'='*40}\n")
        log_file.write(f"{pd.Timestamp.now()}\n")
        log_file.write(f"{'='*40}\n")
        
    for df_index in range(len(dataframes)):
        if dataframes_names:
            print(f"{'*'*40}\n{dataframes_names[df_index]}\n{'*'*40}")
            if log_path:
                log_file.write(f"{'*'*40}\n{dataframes_names[df_index]}\n{'*'*40}\n")
        with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None):
            results = [
                ("First few rows of the dataframe:\n", dataframes[df_index].head()),
                ("Columns in the dataframe:\n", dataframes[df_index].columns),
                ("Number of rows in the dataframe:\n", dataframes[df_index].shape[0]),
                ("Missing values in the dataframe:\n", dataframes[df_index].isnull().sum()),
                ("Number of duplicate rows in the dataframe:\n", dataframes[df_index].duplicated().sum()),
                ("Data types of columns:\n", dataframes[df_index].dtypes),
                ("Number of unique values in each column:\n", dataframes[df_index].nunique()),
            ]
            for label, result in results:
                print(label)
                print(result)
                print()
                if log_path:
                    log_file.write(f"{label}{result}\n\n")
            if log_path:
                log_file.write(f"{'X'*40}\n")
    if log_path:
        log_file.close()

In [4]:
imputation_cohort_var_non_ICU = "cohort_non-icu_imputation"
imputation_cohort_var_ICU = "cohort_icu_imputation"
imputation_cohort_dataset_non_ICU = pd.read_csv(f"./data/cohort/{imputation_cohort_var_non_ICU}.csv.gz")
imputation_cohort_dataset_ICU = pd.read_csv(f"./data/cohort/{imputation_cohort_var_ICU}.csv.gz")

In [73]:
diag_flag = False
lab_flag = False
proc_flag = True
med_flag = False
cohort_output = imputation_cohort_var_non_ICU
version_path = "mimiciv/2.2"
feature_nonicu_imputation(cohort_output,version_path,diag_flag,lab_flag,proc_flag,med_flag)

[EXTRACTING PROCEDURES DATA]
# Unique ICD9 Procedures:   2544
# Unique ICD10 Procedures:  10031

Value counts of each ICD version:
 icd_version
9     446079
10    223107
Name: count, dtype: int64
# Admissions:   229445
Total number of rows:  669186
[SUCCESSFULLY SAVED PROCEDURES DATA]


In [74]:
procedure_df = pd.read_csv(f"./data/features/imputation_non_icu_preproc_proc.csv.gz")
eda_basic([procedure_df], ['Procedures'], './data/summary/')

****************************************
Procedures
****************************************
First few rows of the dataframe:

   subject_id   hadm_id  seq_num   chartdate icd_code  icd_version  \
0    10000032  22595853        1  2180-05-07     5491            9   
1    10000032  22841357        1  2180-06-27     5491            9   
2    10000032  25742920        1  2180-08-06     5491            9   
3    10000068  25022803        1  2160-03-03     8938            9   
4    10000117  27988844        1  2183-09-19  0QS734Z           10   

             admittime            dischtime  \
0  2180-05-06 22:23:00  2180-05-07 17:15:00   
1  2180-06-26 18:27:00  2180-06-27 18:49:00   
2  2180-08-05 23:44:00  2180-08-07 17:50:00   
3  2160-03-03 23:16:00  2160-03-04 06:26:00   
4  2183-09-18 18:10:00  2183-09-21 16:30:00   

                                          long_title proc_time_from_admit  
0                    Percutaneous abdominal drainage      0 days 01:37:00  
1                